In [6]:
import requests
import re
import pandas as pd
from multiprocessing import Pool

import sys
sys.path.append('../')
from src.functions import get_prokaryotic_organisms, get_genes_for_organism, grouper, fetch_info, check_list_length

from random import randint
from time import sleep
import ast

from concurrent.futures import ThreadPoolExecutor
import threading
import csv 
import os
import glob
import tarfile
import s3fs

In [88]:
import requests
import re
import pandas as pd
from multiprocessing import Pool

import os
print(os.getcwd()) 


import sys

from src.functions import get_prokaryotic_organisms, get_genes_for_organism, grouper, import_s3_folder, fetch_info, check_list_length 

from random import randint
from time import sleep
import ast

from concurrent.futures import ThreadPoolExecutor
import threading
import csv 
import os
import glob
import tarfile


import s3fs

import multiprocessing as mp

s3_f_folder = 'KEGG_db/filled'

local_path_f_new = "/home/onyxia/work/KEGG_Pipeline/newly_filled/"

if not os.path.exists(local_path_f_new):
    os.makedirs(local_path_f_new)



def check_if_prot_list_API(page_response_text):

    pattern = re.compile(r'ORGANISM(.*?)NTSEQ', re.DOTALL)
    extracts = re.findall(pattern, page_response_text)
    indic_AA = [True if "AASEQ" in extract else False for extract in extracts]

    return indic_AA




def scrape_genome_API(organism_code):
    genes = get_genes_for_organism(organism_code)
    organism = organism_code
    print(organism_code)
    dict_gene_org= []


    #we iterate over chunks of 10 genes_ids
    for i, group in enumerate(grouper(genes, 10)):
        
        results = {'gene_id': [], 'K0': [], 'Dna_SEQ': [], 'Prot': []}
        
        sleep(randint(20, 40))
        gene_ids, k0_value, Nt_seq, prot = fetch_info_API(group)
              
        results['gene_id'].extend(gene_ids)
        results['K0'].extend(k0_value)
        results['Dna_SEQ'].extend(Nt_seq)
        results['Prot'].extend(prot)

        dict_gene_org.append(results)
    df = pd.DataFrame(dict_gene_org)

    #Sometimes, due to high number of requests, requests get empty page, here we make sure that the dataframe is filled when written
    
    while ((df['K0'].apply(check_list_length)) | (df['Dna_SEQ'].apply(check_list_length)) | (df['Prot'].apply(check_list_length))).any():
        
        missing_info = df[df.apply(lambda row: any(check_list_length(cell) for cell in row[['K0', 'Dna_SEQ', 'Prot']]), axis=1)]
        
        for index, row in missing_info.iterrows():
            gene_ids_list = row['gene_id']

            # Fetch information for missing gene IDs
            sleep(randint(10,20))
            k0_values, dna_seq_values, prot_values = fetch_info_API(gene_ids_list)

            # Update the original DataFrame with the fetched information
            df.at[index, 'K0'] = k0_values
            df.at[index, 'Dna_SEQ'] = dna_seq_values
            df.at[index, 'Prot'] = prot_values

    df.to_parquet(local_path_f_new +organism+ '_genome'   +'.parquet', index= False)
    #upload_s3_csv(s3_f_folder, local_path_f_new + 'gene_' + organism +'_full.csv' )
    return results






/home/onyxia/work/KEGG_Pipeline/notebooks


In [89]:
genomes = ['pant', 'lkf','rbn','psin','sene','smt','nfl','mvz']

genes = get_genes_for_organism('mvz')

In [90]:
genes

['mvz:myaer102_00010',
 'mvz:myaer102_00020',
 'mvz:myaer102_00030',
 'mvz:myaer102_00040',
 'mvz:myaer102_00050',
 'mvz:myaer102_00060',
 'mvz:myaer102_00070',
 'mvz:myaer102_00080',
 'mvz:myaer102_00090',
 'mvz:myaer102_00100',
 'mvz:myaer102_00110',
 'mvz:myaer102_00120',
 'mvz:myaer102_00130',
 'mvz:myaer102_00140',
 'mvz:myaer102_00150',
 'mvz:myaer102_00160',
 'mvz:myaer102_00170',
 'mvz:myaer102_00180',
 'mvz:myaer102_00190',
 'mvz:myaer102_00200',
 'mvz:myaer102_00210',
 'mvz:myaer102_00220',
 'mvz:myaer102_00230',
 'mvz:myaer102_00240',
 'mvz:myaer102_00250',
 'mvz:myaer102_00260',
 'mvz:myaer102_00270',
 'mvz:myaer102_00280',
 'mvz:myaer102_00290',
 'mvz:myaer102_00300',
 'mvz:myaer102_00310',
 'mvz:myaer102_00320',
 'mvz:myaer102_00330',
 'mvz:myaer102_00340',
 'mvz:myaer102_00350',
 'mvz:myaer102_00360',
 'mvz:myaer102_00370',
 'mvz:myaer102_00380',
 'mvz:myaer102_00390',
 'mvz:myaer102_00400',
 'mvz:myaer102_00410',
 'mvz:myaer102_00420',
 'mvz:myaer102_00430',
 'mvz:myaer

In [91]:
genes = genes[:10]
#['mvz:myaer102_00010']

In [92]:
test = fetch_info_API(genes)

[True, True, True, True, True, True, False, False, True, False]


In [93]:
test

(['mvz:myaer102_00010',
  'mvz:myaer102_00020',
  'mvz:myaer102_00030',
  'mvz:myaer102_00040',
  'mvz:myaer102_00050',
  'mvz:myaer102_00060',
  'mvz:myaer102_00070',
  'mvz:myaer102_00080',
  'mvz:myaer102_00090',
  'mvz:myaer102_00100'],
 ['K02313',
  'K02257',
  'K02259',
  'K02275',
  'K02274',
  'K02276',
  '',
  '',
  'K01406',
  ''],
 ['gtggacagcagcgcccaacaactttggcataatctcctagagcgtctaaaattactattaacccgtcccgctttcgagacttggtttcaaacagcgacggttaaagaatggcaaaacgatcgcttagtaattcaggctgccaatccctttattcttaaccatcttcagaaaaattatctgtcaacaattgccgaagtagtgcaggatatcgtcggttatcccgtcgagattcaactaacggcacaacagggcgatttgatcgctattttccagcctaacagtagcttagaatcagaattatcccccagtaatcaacttaaccctaaatataatttttcccgttttgttgtcgggccaaccaatcgtatggcccacgcggccgctttagcggtggcggaattacccggacgcgaatttaacccgctttttctctgtggtggggtgggattaggtaaaacccatttaatgcaggccatcggtcattatcgtctggaattatatcctcaatctagggttttttatgtatcgacggaacaatttacgaatgatttaatcaccgctatccgtcaagatagtatggaaagttttcgtaatcattatcgtcatgcggacatattattagtggatgatctgcaatttattgaaggaaaaga

In [66]:
#regex
pattern_KO = re.compile(r'ORTHOLOGY\s+(K\d{5})', re.DOTALL)
pattern_seq = re.compile(r'NTSEQ\s+\d+(.*?)///', re.DOTALL)
pattern_extract = re.compile(r'ENTRY(.*?)ORGANISM', re.DOTALL)

In [85]:
#gene_ids
gene_ids = [gene for gene in genes if gene is not None]
gene_group_url = '+'.join(gene_ids)
gene_page_response = requests.get(f'https://rest.kegg.jp/get/{gene_group_url}')

In [86]:
extracts = re.findall(pattern_extract, gene_page_response.text)
indic_k0 = [True if "ORTHOLOGY" in extract else False for extract in extracts]

In [87]:
indic_k0

[True, True, True, True, True, True, False, False, True, False]

In [69]:
ko= re.findall(pattern_KO,gene_page_response.text)

In [70]:
ko

['K02313']

In [ ]:
indic_k0 = [True if "KO" in extract else False for extract in extracts]
index_A = 0
k0_value = []
for element_B in indic_k0:
    if element_B and index_A < len(match):
        k0_value.append(match[index_A])
        index_A += 1
    else:
        k0_value.append('')

In [37]:
gene_page_response.text

'ENTRY       myaer102_00010    CDS       T06032\nSYMBOL      dnaA\nNAME        (GenBank) chromosomal replication initiator protein DnaA\nORTHOLOGY   K02313  chromosomal replication initiator protein\nORGANISM    mvz  Microcystis viridis\nPATHWAY     mvz02020  Two-component system\nBRITE       KEGG Orthology (KO) [BR:mvz00001]\n             09130 Environmental Information Processing\n              09132 Signal transduction\n               02020 Two-component system\n                myaer102_00010 (dnaA)\n             09180 Brite Hierarchies\n              09182 Protein families: genetic information processing\n               03032 DNA replication proteins [BR:mvz03032]\n                myaer102_00010 (dnaA)\n               03036 Chromosome and associated proteins [BR:mvz03036]\n                myaer102_00010 (dnaA)\n            DNA replication proteins [BR:mvz03032]\n             Prokaryotic type\n              DNA Replication Initiation Factors\n               Initiation factors (bacte

In [ ]:



# Finding the KO
match = re.findall(pattern_KO, gene_page_response.text)
extracts = re.findall(pattern_extract, gene_page_response.text)
indic_k0 = [True if "KO" in extract else False for extract in extracts]
index_A = 0
k0_value = []
for element_B in indic_k0:
    if element_B and index_A < len(match):
        k0_value.append(match[index_A])
        index_A += 1
    else:
        k0_value.append('')
    
#Finding the RNA NT sequence
Nt_seq = re.findall(pattern_seq, gene_page_response.text)
Nt_seq = [seq.replace(' ', '').replace('\n', '') for seq in Nt_seq]

#Finding whether it is a protein or not
prot = check_if_prot_list_API(gene_page_response.text)
return gene_ids, k0_value, Nt_seq, prot